# Chapter 4 **Numerical Computing with Julia**


----
## Traversing matrices efficiency

In [1]:
using BenchmarkTools;

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1242


In [6]:
# Juliaの中ではcolumn優先でデータを保持するので，行ごとよりも列ごとに演算したほうが高速になる。
function sum_by_col(x)
    s = zero(eltype(x))
    for j in 1:size(x, 2)
        for i in 1:size(x, 1)
            s += x[i, j]
        end
    end
    s
end

function sum_by_row(x)
    s = zero(eltype(x))
    for i in 1:size(x, 1)
        for j in 1:size(x, 2)
            s += x[i, j]
        end
    end
    s
end

# Benchmark
x = rand(10^4, 10^4)
@time sum_by_row(x) ;@time sum_by_col(x) ;@time sum(x) 

  0.721675 seconds (26.21 k allocations: 1.400 MiB)
  0.106646 seconds (26.21 k allocations: 1.400 MiB)
  0.040035 seconds (5 allocations: 176 bytes)


5.00006775138412e7

----
## Executing loops efficiency with conditional statements

In [ ]:
# loopの効率のよい実行方法
x = randn(10^6)

**type1**  
一行で書くことがdきるが，あまりパフォーマンスは良くない。

In [7]:
@time sum(v for v in x if v > 0)

  0.132494 seconds (127.94 k allocations: 6.226 MiB)


5.000067751384889e7

**type2**  
loop + 関数化  
基本的に関数にしたほうが高速になる

In [8]:
function possum1(x)
    s = zero(eltype(x)) # オブジェクトの型を自動的に判別してくれる。
    for v in x
        if(v > 0)
            s += v
        end
    end
    s
end
possum1(x)

5.000067751384889e7

**type3**  
loopの中のif(conditional statement)は効率が悪いので，ifelseで置き換える。`if`の中に入っている計算が軽いものであれば，コンパイル後のbranchingを避けたほうが高速に演算される(?)。

In [10]:
# type 3
function possum2a(x)
    s = zero(eltype(x)) # オブジェクトの型を自動的に判別してくれる。
    for v in x
        s += ifelse(v > 0, v, zero(s))
    end
    s
end
# precompile
possum2a(x);

5.000067751384889e7

**type4**  
`@simd` macroを使うやり方。使用方法は簡単で，loopの直前にマクロを挿入するだけ。SIMDプロセッサは特定の条件下でloop処理を高速にしてくれる。ただしSIMDの高速化性能はハードにも依存するようだ。

In [14]:
# type 4
# add @simd macro
function possum2b(x)
    s = zero(eltype(x)) 
    @simd for v in x
        s += ifelse(v > 0, v, zero(s)) 
    end
    s
end

# poor implemention
function possum2c(x)
    s = zero(eltype(x)) 
    @simd for v in x
        s += ifelse(v > 0, v, 0)　# Integerにも typeof(v)にも，どちらにもなりうる
    end
    s　# 最終的に返ってくる`s`の型判定が不安定なので，速度が遅い。
end

function possum2d(x::AbstractArray{T}) where T 
    # `where T` is shorthand for where T<:any. つまり何らかの型を指定するということ。
    # この例では関数に与えたArrayから型を読み取っている。
    s = zero(T) 
    @simd for v in x
        s += ifelse(v > 0, v, zero(T))
    end
    s
end
# precompile
possum2b(x)
possum2c(x)
possum2d(x);

In [15]:
@btime possum2a(x)
@btime possum2b(x)
@btime possum2c(x)
@btime possum2d(x);

  95.068 ms (1 allocation: 16 bytes)
  32.828 ms (1 allocation: 16 bytes)
  96.920 ms (1 allocation: 16 bytes)
  32.539 ms (1 allocation: 16 bytes)


5.000067751384136e7

**type5**  
三項演算子`a ? b : c`を使う場合。

In [20]:
function possum2e(x::AbstractArray{T}) where T 
    # `where T` is shorthand for where T<:any. つまり何らかの型を指定するということ。
    # この例では関数に与えたArrayから型を読み取っている。
    s = zero(T) 
    @simd for v in x
        s += v > 0 ? v : zero(T)
    end
    s
end
possum2e(x);
@btime possum2e(x);

  32.385 ms (1 allocation: 16 bytes)


----
## Generating full factorial designs

要するにRでいうところの`expand.grid()`関数を実現する。

In [22]:
using DataFrames

┌ Info: Recompiling stale cache file /Users/takuizum/.julia/compiled/v1.2/DataFrames/AR9oZ.ji for DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1240


In [35]:
v = vec(collect(Base.product([1,2],["A", "B"], [1.0, 2.2])))
DataFrame(collect.(collect(zip(v...))))

,x1,x2,x3
,Int64,String,Float64
1,1,A,1.0
2,2,A,1.0
3,1,B,1.0
4,2,B,1.0
5,1,A,2.2
6,2,A,2.2
7,1,B,2.2
8,2,B,2.2


In [46]:
function expandgrid(levels...)
    lengths = length.(levels)
    inner = 1
    outer = prod(lengths)
    grid = []
    for i in 1:length(levels)
        outer = div(outer, lengths[i])
        push!(grid, repeat(levels[i], inner = inner, outer = outer))
        inner *= lengths[i]
    end
    DataFrame(hcat(Tuple(grid)...))
end

expandgrid([1,2],["A", "B"], [1.0, 2.2])

,x1,x2,x3
,Any,Any,Any
1,1,A,1.0
2,2,A,1.0
3,1,B,1.0
4,2,B,1.0
5,1,A,2.2
6,2,A,2.2
7,1,B,2.2
8,2,B,2.2


----
## Running Monte Carlo simulations

内包表記と`map`とベクタライズによる計算時間とメモリ消費量の違い

In [49]:
using OnlineStats, Random

┌ Info: Precompiling OnlineStats [a15396b6-48d5-5d58-9928-6d29437db91e]
└ @ Base loading.jl:1242


In [79]:
function simwalk()
    jumps = 0
    distance = 0
    while true
        jumps += 1
        distance += rand()
        distance >= 1.0 && return jumps
    end
end

function increasemental(n)
    s = Mean()
    for i in 1:n
        fit!(s, simwalk() )
    end
    value(s)
end
# precompile
n = 10^8
increasemental(1);

### First method  
最もシンプルな書き方。内包表記でループした上で，ベクタライズして，その平均を取る。中間で`simwalk`による計算結果を一時的に保持している。

In [70]:
@time mean([simwalk() for i in 1:n]);

  2.023130 seconds (48.08 k allocations: 765.362 MiB, 0.35% gc time)


### Second method  
`map`関数内で関数を指定し，入力変数をベクトル化して計算。関数型のプログラミングアプローチに近い。

In [71]:
@time mean(map(x -> simwalk(), 1:n));

  2.048181 seconds (56.32 k allocations: 765.785 MiB, 2.93% gc time)


### Third method  
1つ目の方法とよく似ているが，計算結果を直接`mean`関数に与えているため，途中の計算結果を保持しない。

In [72]:
@time mean(simwalk() for i in 1:n);

  1.601154 seconds (31.49 k allocations: 1.679 MiB)


### Fourth method  
`OnlineStats`パッケージの関数を利用して，最も効率的に実行できる。

In [81]:
@time increasemental(n);

  1.611675 seconds (5 allocations: 176 bytes)


### Fifth method
関数化すれば，単純なループ構造でも非常に効率がよい。

In [77]:
function simwalk(n)
    jumps = 0
    for i in 1:n
        distance = 0.0
         while true
             jumps += 1
             distance += rand()
             distance >= 1.0 && break
         end
    end
    jumps / n
end
simwalk(1);
@time simwalk(n);

  1.169228 seconds (5 allocations: 176 bytes)
